In [ ]:
https://www.fia.com/documents/championships/fia-formula-one-world-championship-14/season/season-2019-971/event/Abu%20Dhabi%20Grand%20Prix
https://www.fia.com/documents/championships/fia-formula-one-world-championship-14/season/season-2020-1059/event/Abu%20Dhabi%20Grand%20Prix
https://www.fia.com/documents/championships/fia-formula-one-world-championship-14/season/season-2021-1108/event/Abu%20Dhabi%20Grand%20Prix

In [165]:
my_dict = {
    'Abu Dhabi': [19, 20, 21],
    'Australian': [19, 20],
    'Austrian': [19, 20, 21],
    'Azerbaijan': [19, 21],
    'Bahrain': [19, 20, 21],
    'Belgian': [19, 20, 21],
    'Brazilian': [19, 21],
    'British': [19, 20, 21],
    'Dutch': [21],
    'Canadian': [19],
    'Chinese': [19],
    'Eifel': [20],
    'Emilia Romagna': [20, 21],
    'Formula 1 70th Anniversary': [20],
    'French': [19, 21],
    'German': [19],
    'Hungarian': [19, 20, 21],
    'Italian': [19, 20, 21],
    'Japanese': [19],
    'Mexican': [19, 21],
    'Monaco': [19, 21],
    'Portuguese': [20, 21],
    'Qatar': [21],
    'Russian': [19, 20, 21],
    'Sakhir': [20],
    'Saudi Arabia': [21],
    'Singapore': [19],
    'Spanish': [19, 21],
    'Styrian': [20, 21],
    'Turkish': [20, 21],
    'Tuscan': [20],
    'United States': [19, 21]}

In [166]:
import pandas as pd
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup

In [42]:
# 2019 RACE FILES

races_19 = [k for k, v in my_dict.items() if 19 in v]
races_19 = [(k,"https://www.fia.com/documents/championships/fia-formula-one-world-championship-14/season/season-2019-971/event/" + k.replace(" ","%20") + "%20Grand%20Prix") for k in races_19]

folder_location = '../Downloaded Data/2019'
if not os.path.exists(folder_location):os.mkdir(folder_location)

for (race, url) in races_19: 
    folder_location = '../Downloaded Data/2019/'+race
    if not os.path.exists(folder_location):os.mkdir(folder_location)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")     
    for link in soup.select("a[href$='.pdf']"):
        filename = os.path.join(folder_location,link['href'].split('/')[-1])
        with open(filename, 'wb') as f:
            f.write(requests.get(urljoin(url,link['href'])).content)



In [168]:
# 2020 RACE FILES

races_20 = [k for k, v in my_dict.items() if 20 in v]
races_20 = [(k,"https://www.fia.com/documents/championships/fia-formula-one-world-championship-14/season/season-2020-1059/event/" + k.replace(" ","%20") + "%20Grand%20Prix") for k in races_20]


folder_location = '../Downloaded Data/2020'
if not os.path.exists(folder_location):os.mkdir(folder_location)

for (race, url) in races_20: 
    folder_location = '../Downloaded Data/2020/'+race
    if not os.path.exists(folder_location):os.mkdir(folder_location)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")     
    for link in soup.select("a[href$='.pdf']"):
        filename = os.path.join(folder_location,link['href'].split('/')[-1])
        with open(filename, 'wb') as f:
            f.write(requests.get(urljoin(url,link['href'])).content)

In [170]:
# 2021 RACE FILES

races_21 = [k for k, v in my_dict.items() if 21 in v]
races_21 = [(k,"https://www.fia.com/documents/championships/fia-formula-one-world-championship-14/season/season-2021-1108/event/" + k.replace(" ","%20") + "%20Grand%20Prix") for k in races_21]


folder_location = '../Downloaded Data/2021'
if not os.path.exists(folder_location):os.mkdir(folder_location)

for (race, url) in races_21: 
    folder_location = '../Downloaded Data/2021/'+race
    if not os.path.exists(folder_location):os.mkdir(folder_location)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")     
    for link in soup.select("a[href$='.pdf']"):
        filename = os.path.join(folder_location,link['href'].split('/')[-1])
        with open(filename, 'wb') as f:
            f.write(requests.get(urljoin(url,link['href'])).content)

In [158]:
# 2019 ALL TIMINGS

links = []
url = "https://www.fia.com/f1-archives?season=971"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")     
for link in soup.findAll('a'):
    links.append(link.get('href'))

links = [link for link in links if "/events/fia-formula-one-world-championship" in link]
links = ["https://www.fia.com" + link if "https://www.fia.com" not in link else link for link in links]
links = [link for link in links if "2019" in link]
links = [link.replace("https://admin.fia.com","") for link in links]

master_dict = {}
key = "season"
i = 1
for link in links:
    if 'https://www.fia.com/championship/events/fia-formula-one-world-championship/season-2019/' in link:
        key = link.replace('https://www.fia.com/championship/events/fia-formula-one-world-championship/season-2019/','')
        master_dict.update({key:[link]})
    elif i == 1:
        key = "season"
        master_dict.update({key:[link]})   
    else:
        my_list = master_dict[key]
        my_list.append(link)
        master_dict.update({key:my_list})
    i = 0
    
urls = {a:[link for link in b if "eventtiming" in link] for (a,b) in master_dict.items()}
del(urls["season"])
urls.update({'austrian-grand-prix': ['https://www.fia.com/eventtiming-information']})
urls


{'australian-grand-prix': ['https://www.fia.com/events/fia-formula-one-world-championship/season-2019/eventtiming-information-1'],
 'bahrain-grand-prix': ['https://www.fia.com/events/fia-formula-one-world-championship/season-2019/eventtiming-information-3'],
 'chinese-grand-prix': ['https://www.fia.com/events/fia-formula-one-world-championship/season-2019/eventtiming-information-7'],
 'azerbaijan-grand-prix-0': ['https://www.fia.com/events/fia-formula-one-world-championship/season-2019/eventtiming-information-8'],
 'spanish-grand-prix': ['https://www.fia.com/events/fia-formula-one-world-championship/season-2019/eventtiming-information-10'],
 'monaco-grand-prix': ['https://www.fia.com/events/fia-formula-one-world-championship/season-2019/eventtiming-information-11'],
 'canadian-grand-prix': ['https://www.fia.com/events/fia-formula-one-world-championship/season-2019/eventtiming-information-13'],
 'french-grand-prix': ['https://www.fia.com/events/fia-formula-one-world-championship/season-

In [164]:
folder_location = '../Downloaded Data/2019'
if not os.path.exists(folder_location):os.mkdir(folder_location)

for (race, url) in urls.items():
    race = race.replace("-0","").replace("-grand-prix","").replace("-"," ").title()
    folder_location = '../Downloaded Data/2019/'+race
    if not os.path.exists(folder_location):os.mkdir(folder_location)
    response = requests.get(url[0])
    soup= BeautifulSoup(response.text, "html.parser")     
    for link in soup.select("a[href$='.pdf']"):
        filename = os.path.join(folder_location,link['href'].split('/')[-1])
        with open(filename, 'wb') as f:
            f.write(requests.get(urljoin(url[0],link['href'])).content)

In [ ]:
https://www.fia.com/f1-archives?season=1059